<a href="https://colab.research.google.com/github/ani-34/footypredictor/blob/master/Transfermarkt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.layers import Dense
from tensorflow.keras import activations
from tensorflow.keras import callbacks
from tensorflow.keras import models

In [3]:
cols = pd.read_csv("data.csv", nrows = 0).columns.tolist()
remove_cols = [0, 1, 2, 4, 5, 6, 9, 10, 13, 19, 20, 22, 23, 24, 25, 26, 27, 88]
cats = ["Work Rate", "Position"]
for i in range(28, 54):
    remove_cols.append(i)

In [4]:
def currency_to_number(np_array):
    np_array = np_array.str.replace("€", "")
    np_array = np_array.replace({'K': '*1e3', 'M': '*1e6'}, 
                                regex=True).map(pd.eval).astype(float)
    return np_array

In [13]:
def create_df(file_name):
    _df = pd.read_csv(file_name, delim_whitespace=False, names=cols, skiprows=1)
    names = _df["Name"]
    _df.drop(_df.columns[remove_cols],axis=1,inplace=True)
    _df["Preferred Foot"].replace("Left", 1, inplace=True)
    _df["Preferred Foot"].replace("Right", 0, inplace=True)
    _df["Position"].replace("RCB", "CB", inplace=True)
    _df["Position"].replace("LCB", "CB", inplace=True)
    _df["Position"].replace("RDM", "CDM", inplace=True)
    _df["Position"].replace("LDM", "CDM", inplace=True)
    _df["Position"].replace("RCM", "CM", inplace=True)
    _df["Position"].replace("LCM", "CM", inplace=True)
    _df["Position"].replace("RAM", "CAM", inplace=True)
    _df["Position"].replace("LAM", "CAM", inplace=True)
    _df["Position"].replace("RF", "RW", inplace=True)
    _df["Position"].replace("LF", "LW", inplace=True)
    _df["Position"].replace("RS", "ST", inplace=True)
    _df["Position"].replace("LS", "ST", inplace=True)
    _df = pd.get_dummies(_df, prefix=cats, columns=cats)
    _df["Value"] = currency_to_number(_df["Value"])
    _df["Wage"] = currency_to_number(_df["Wage"])

    return _df, names

df, _ = create_df("data.csv")
mins = {}
maxs = {}
for a in df.columns:
    if a not in cats:
        mins[a] = df[a].min()
        maxs[a] = df[a].max()

def scale_data(_df):
    for a in _df.columns:
        if a not in cats:
            _df[a] = (_df[a] - mins[a]) / (maxs[a] - mins[a])
    return _df

df = scale_data(df)
corr_matrix = df.corr()
print(corr_matrix)
drop_cols = []
for i in range(0, len(corr_matrix["Value"])):
    if abs(corr_matrix["Value"][i]) < 0.15 and "Position" not in df.columns[i]:
        drop_cols.append(i)
    if "GK" in df.columns[i]:
        df[df.columns[i]] = df[df.columns[i]] * df["Position_GK"]
df = df.drop(df.columns[drop_cols], axis=1).fillna(0)
df[df < 0] = 0
df[df > 1] = 1

df

                   Age   Overall  ...  Position_RWB  Position_ST
Age           1.000000  0.452350  ...     -0.011194    -0.010962
Overall       0.452350  1.000000  ...     -0.015882    -0.001247
Potential    -0.253312  0.660939  ...     -0.015281     0.003030
Value         0.075022  0.627085  ...     -0.005763     0.018768
Wage          0.141145  0.571926  ...     -0.003571     0.018190
...                ...       ...  ...           ...          ...
Position_RB   0.009863 -0.007191  ...     -0.019142    -0.111793
Position_RM  -0.039260  0.009569  ...     -0.017774    -0.103801
Position_RW  -0.042960  0.020301  ...     -0.010198    -0.059557
Position_RWB -0.011194 -0.015882  ...      1.000000    -0.028040
Position_ST  -0.010962 -0.001247  ...     -0.028040     1.000000

[67 rows x 67 columns]


,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,ShotPower,Stamina,LongShots,Aggression,Positioning,Vision,Penalties,Composure,GKPositioning,Position_CAM,Position_CB,Position_CDM,Position_CF,Position_CM,Position_GK,Position_LB,Position_LM,Position_LW,Position_LWB,Position_RB,Position_RM,Position_RW,Position_RWB,Position_ST
0,1.000000,0.978723,0.932489,1.000000,1.00,0.75,0.75,0.897727,1.000000,0.733333,0.965116,0.953488,1.000000,0.988636,1.000000,0.928571,1.000000,0.929412,0.880952,0.939024,0.986667,0.892473,0.714286,1.000000,0.440476,0.989247,1.000000,0.804598,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.000000,0.978723,0.649789,0.716814,1.00,0.75,1.00,0.897727,0.989247,0.944444,0.860465,0.965116,0.903226,0.852273,0.802198,0.809524,0.978022,0.905882,0.940476,0.890244,1.000000,1.000000,0.904762,0.989011,0.619048,1.000000,0.857143,0.919540,0.989247,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.958333,0.957447,1.000000,0.513274,1.00,1.00,1.00,0.840909,0.913978,0.644444,0.895349,0.930233,0.989247,0.931818,0.923077,0.821429,0.989011,0.964706,0.928571,1.000000,0.973333,0.838710,0.821429,0.868132,0.535714,0.935484,0.916667,0.873563,0.978495,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.937500,0.957447,0.607595,0.460177,0.75,0.50,0.00,0.136364,0.118280,0.188889,0.500000,0.104651,0.150538,0.170455,0.175824,0.500000,0.406593,0.529412,0.547619,0.560976,0.920000,0.311828,0.369048,0.098901,0.321429,0.107527,0.690476,0.402299,0.698925,0.977528,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.937500,0.936170,0.860759,0.628319,0.75,1.00,0.75,1.000000,0.860215,0.566667,0.988372,0.906977,0.881720,0.897727,0.879121,0.976190,0.945055,0.776471,0.761905,0.792683,0.933333,0.956989,0.928571,0.967033,0.773810,0.913978,1.000000,0.850575,0.913978,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,0.020833,0.361702,0.000506,0.001770,0.00,0.25,0.25,0.329545,0.387097,0.400000,0.488372,0.244186,0.408602,0.272727,0.340659,0.428571,0.417582,0.494118,0.535714,0.560976,0.373333,0.440860,0.333333,0.384615,0.416667,0.397849,0.500000,0.436782,0.451613,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18203,0.020833,0.319149,0.000506,0.001770,0.00,0.25,0.25,0.204545,0.537634,0.533333,0.418605,0.372093,0.376344,0.295455,0.186813,0.190476,0.384615,0.341176,0.321429,0.292683,0.253333,0.419355,0.369048,0.428571,0.428571,0.473118,0.273810,0.436782,0.419355,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18204,0.020833,0.404255,0.000506,0.001770,0.00,0.50,0.25,0.227273,0.408602,0.466667,0.360465,0.395349,0.440860,0.363636,0.263736,0.226190,0.428571,0.682353,0.678571,0.439024,0.346667,0.462366,0.511905,0.461538,0.250000,0.494624,0.392857,0.574713,0.408602,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18205,0.020833,0.382979,0.000506,0.001770,0.00,0.50,0.25,0.443182,0.516129,0.388889,0.406977,0.418605,0.505376,0.318182,0.318681,0.273810,0.516484,0.576471,0.571429,0.463415,0.000000,0.666667,0.333333,0.340659,0.261905,0.451613,0.440476,0.517241,0.462366,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
y = df["Value"]
X = df.drop(columns=["Value"])
xtrain, xvaltest, ytrain, yvaltest = train_test_split(X, y, test_size=0.3)
xval, xtest, yval, ytest = train_test_split(xvaltest, yvaltest, test_size=0.5)

weight_neuron = []

train_len = len(xtrain)
test_len = len(xtest)
y_len = len(ytrain)

class CustomCallback(callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        temp = self.model.layers[2].get_weights()[0].tolist()
        weight_neuron.append(temp)

model = models.Sequential()
model.add(Dense(85,activation="sigmoid",input_dim=len(X.columns)))
model.add(Dense(50,activation="sigmoid"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",
            loss="mean_squared_error")

weight_neuron = []

history_train = model.fit(xtrain, ytrain,
                batch_size=32, epochs=100,
                validation_data=(xval, yval),
                callbacks=[CustomCallback()],
                )


Epoch 1/100
399/399 [==============================] - 15s 35ms/step - loss: 0.0041 - val_loss: 0.0025
Epoch 2/100
399/399 [==============================] - 2s 5ms/step - loss: 0.0022 - val_loss: 0.0025
Epoch 3/100
399/399 [==============================] - 1s 3ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 4/100
399/399 [==============================] - 1s 3ms/step - loss: 0.0019 - val_loss: 0.0020
Epoch 5/100
399/399 [==============================] - 1s 3ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 6/100
399/399 [==============================] - 1s 3ms/step - loss: 8.0637e-04 - val_loss: 6.4751e-04
Epoch 7/100
399/399 [==============================] - 1s 3ms/step - loss: 6.1080e-04 - val_loss: 5.5286e-04
Epoch 8/100
399/399 [==============================] - 1s 3ms/step - loss: 5.3835e-04 - val_loss: 5.7172e-04
Epoch 9/100
399/399 [==============================] - 1s 3ms/step - loss: 4.8422e-04 - val_loss: 4.2820e-04
Epoch 10/100
399/399 [==============================] - 1

In [22]:
new_df, names = create_df("test_data.csv")
new_df = scale_data(new_df)
for col in new_df.columns:
    if col not in X.columns:
        new_df.drop(columns=[col], inplace=True)
for i in range(0, len(X.columns)):
    if X.columns[i] not in new_df.columns:
        index = new_df.index
        number_of_rows = len(index)
        new_df.insert(i, X.columns[i], np.zeros(number_of_rows))

predictions = model.predict(new_df)
for i in range(0, len(predictions)):
    val = (predictions[i] * (maxs["Value"] - mins["Value"])) + mins["Value"]
    prefix = ""
    if val > 1000000:
        prefix = "M"
        val = val / 1000000
    elif val > 1000:
        prefix = "K"
        val = val / 1000
    print(names[i] + " is worth €" + str(val[0].round(2)) + prefix)

Ani is worth €17.23M
Noor is worth €19.82M
Luke is worth €50.3M
Fernando is worth €50.8M
Noah is worth €59.89M
Will is worth €26.85M
Min is worth €42.26M
Random goalkeeper from the other day is worth €35.69K
L. Messi is worth €95.96M
De Gea is worth €57.4M
Sergio Ramos is worth €53.74M
H. Kane is worth €59.81M
E. Hazard is worth €83.62M
